<a href="https://colab.research.google.com/github/Lin1225/siam_mot_github/blob/main/siam_mot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [ ]:
!git clone https://github.com/amazon-research/siam-mot.git

In [ ]:
!pip3 install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
%cd siam-mot
!pip3 install -r requirements.txt

In [ ]:
%cd /content/
!pip3 install ninja yacs cython matplotlib tqdm opencv-python
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!python3 setup.py build_ext install

%cd /content/
!git clone https://github.com/mcordts/cityscapesScripts.git
%cd cityscapesScripts/
!python3 setup.py build_ext install

%cd /content/
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!python3 setup.py install --cuda_ext --cpp_ext

# %cd /content/
# !git clone https://github.com/facebookresearch/maskrcnn-benchmark.git
# %cd maskrcnn-benchmark
# !python3 setup.py build develop


In [ ]:
%cd /content/
!git clone https://github.com/facebookresearch/maskrcnn-benchmark.git

%cd maskrcnn-benchmark
!perl -i -pe 's/AT_CHECK/TORCH_CHECK/' $"maskrcnn_benchmark/csrc/cuda"/deform_pool_cuda.cu $"maskrcnn_benchmark/csrc/cuda"/deform_conv_cuda.cu
!python3 setup.py build develop


In [ ]:
!pip3 install ffmpeg-python

# modified demo.py

In [ ]:
%%file /content/siam-mot/demos/demo.py

import sys
sys.path.append('/content/siam-mot/')

import logging
import argparse
import os

from demos.demo_inference import DemoInference
from demos.utils.vis_generator import VisGenerator
from demos.utils.vis_writer import VisWriter
from demos.video_iterator import build_video_iterator

parser = argparse.ArgumentParser(" SiamMOT Inference Demo")
parser.add_argument('--demo-video', metavar="FILE", type=str,
                    required=True)
parser.add_argument('--track-class', type=str, choices=('person', 'person_vehicle'),
                    default='person_vehicle',
                    help='Tracking person or person/vehicle jointly')
parser.add_argument("--dump-video", type=bool, default=True,
                    help="Dump the videos as results")
parser.add_argument("--vis-resolution", type=int, default=1080)
parser.add_argument("--output-path", type=str, default=None,
                    help='The path of dumped videos')


if __name__ == '__main__':
    args = parser.parse_args()

    logging.basicConfig(format='%(asctime)s,%(msecs)d %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',
                        datefmt='%Y-%m-%d:%H:%M:%S',
                        level=logging.INFO)

    # Build visulization generator and writer
    vis_generator = VisGenerator(vis_height=args.vis_resolution)
    vis_writer = VisWriter(dump_video=args.dump_video,
                           out_path=args.output_path,
                           file_name=os.path.basename(args.demo_video))

    # Build demo inference
    tracker = DemoInference(track_class=args.track_class,
                            vis_generator=vis_generator,
                            vis_writer=vis_writer)

    # Build video iterator for inference
    video_reader = build_video_iterator(args.demo_video)

    results = list(tracker.process_frame_sequence(video_reader()))

    if args.dump_video:
        vis_writer.close_video_writer()




# get video

In [ ]:
%cd  /content/siam-mot
!mkdir -p demos/demo_vis
!wget https://github.com/Lin1225/siam_mot_github/blob/main/person_car.mp4?raw=true
!mv person_car.mp4?raw=true person_car.mp4

# Run

In [ ]:
!python3 demos/demo.py --demo-video "./person_car.mp4"